In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [5]:
design.delete_all_components()

In [6]:
design.get_chip_z('Q_chip')

'5um'

In [3]:
design.chips['C_chip'].size['size_x']='8mm'
design.chips['C_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['size_x']='8mm'
design.chips['Q_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['center_z']='12um'

# Resonator length

In [4]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [5]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.35
    resonator_length_2.append(a)
print(resonator_length_2)


[8.103254683870285, 7.874788339841421, 7.658346540209131]


# Flipchip

In [6]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-150}um",
                                              pos_y=f"{q0y+140}um",
                                              pad_width="150um",
                                              pad_height="30um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [7]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))

In [8]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="C_chip",
                                    down_length='300um',
                                    fillet='99um',
                                    coupling_space='5um',
                                    mirror=True,
                                    open_termination=True))



In [9]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [10]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [11]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')

# EPR Analysis

In [12]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [13]:
hfss.start()

INFO 10:30PM [connect_project]: Connecting to Ansys Desktop API...
INFO 10:30PM [load_ansys_project]: 	Opened Ansys App
INFO 10:30PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 10:30PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project25
INFO 10:30PM [connect_design]: No active design found (or error getting active design).
INFO 10:30PM [connect]: 	 Connected to project "Project25". No design detected


True

In [14]:
hfss.activate_ansys_design("flip12_V1", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','clt0','rr0','fl0','fl1'],
                    [('clt0','prime_end')],
                    box_plus_buffer=False)

10:30PM 24s WARNING [activate_ansys_design]: The design_name=flip12_V1 was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 10:30PM [connect_design]: 	Opened active design
	Design:    flip12_V1 [Solution type: Eigenmode]
WARNING 10:30PM [connect_setup]: 	No design setup detected.
WARNING 10:30PM [connect_setup]: 	Creating eigenmode default setup.
INFO 10:30PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [15]:
hfss.close()

True

# LOM Analysis

In [16]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [17]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='6.99',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '6.99',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [19]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 11:54PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpuigxe0rg.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 6990000000, Maxwell, 1, False
INFO 11:54PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmptv8lqjkn.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 6990000000, Maxwell, 1, False
INFO 11:54PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpctrh3lbk.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 6990000000, Maxwell, 2, False
INFO 11:54PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp0lbdipq_.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 6990000000, Maxwell, 3, False
INFO 11:54PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpe97s7ba0.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 6990000000, Maxwell, 4, False
INFO 11:54

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,307.080,-88.41600,-86.9830,-67.04000
pad_bot_Q0,-88.416,121.38000,-31.7770,-0.13074
pad_top_Q0,-86.983,-31.77700,121.0500,-1.45480
quad_coupler_c0,-67.040,-0.13074,-1.4548,68.82800


In [21]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 10.39
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.171403 [GHz]
EC 248.262676 [MHz]
EJ 14.854145 [GHz]
alpha -280.986895 [MHz]
dispersion 1.898121 [KHz]
Lq 10.995564 [nH]
Cq 78.023118 [fF]
T1 54426.789641 [us]

**Coupling Properties**

tCqbus1 -0.662518 [fF]
gbus1_in_MHz -13.172254 [MHz]
χ_bus1 -0.003808 [MHz]
1/T1bus1 2.924202 [Hz]
T1bus1 54426.789641 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.607285,294.705308,14.854145,-338.244386,10.811942,[-62.93511373214183],[-0.12009722309887189],0.120097,62.935114
2,5.437388,276.068839,14.854145,-315.102194,5.675829,[-12.08820394566301],[-0.0039048636920223827],0.003905,12.088204
3,5.288963,260.347871,14.854145,-295.753479,3.122963,[-13.225063487344785],[-0.004186973620470301],0.004187,13.225063
4,5.272151,258.599879,14.854145,-293.611883,2.912347,[-14.840394657176134],[-0.005207030480172718],0.005207,14.840395
5,5.239691,255.243545,14.854145,-289.50526,2.541763,[-10.789583451888626],[-0.002687040140357115],0.002687,10.789583
6,5.207594,251.948893,14.854145,-285.481103,2.217951,[-11.89499987083967],[-0.0031891764840697796],0.003189,11.895000
7,5.188772,250.02799,14.854145,-283.13807,2.045973,[-12.410013388732565],[-0.003423309218659942],0.003423,12.410013
8,5.175372,248.665516,14.854145,-281.477612,1.931028,[-13.924388699059058],[-0.004267255623605281],0.004267,13.924389
9,5.180875,249.224528,14.854145,-282.158742,1.977501,[-12.846908985557722],[-0.003647217387915635],0.003647,12.846909
10,5.171403,248.262676,14.854145,-280.986895,1.898121,[-13.172254342714332],[-0.0038075112368112533],0.003808,13.172254


In [22]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
